<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/VAE_Gumbel_Softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import torch.utils.data

from torch import nn, optim
from torchvision import datasets, transforms
from torchvision.utils import save_image

import torch.nn.functional as F
import numpy as np
import pandas as pd
import math


In [2]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
from torch.utils.data import DataLoader
# Load the MNIST dataset
mnist_dataset_train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
# Load the MNIST dataset
mnist_dataset_test = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())


batch_size = 128
train_loader = torch.utils.data.DataLoader(mnist_dataset_train,batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_dataset_test,batch_size=batch_size, shuffle=True)

# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
#     batch_size=128, shuffle=True)

# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
#     batch_size=128, shuffle=True)

In [6]:
# Code to implement VAE-gumple_softmax in pytorch

# The code has been modified from pytorch example vae code and inspired by the origianl \
# tensorflow implementation of gumble-softmax by Eric Jang.




def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape).to(device)

    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)


def gumbel_softmax(logits, temperature, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)

    if not hard:
        return y.view(-1, latent_dim * categorical_dim)

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    return y_hard.view(-1, latent_dim * categorical_dim)




In [20]:
class VAE_gumbel(nn.Module):
    def __init__(self, temp):
        super(VAE_gumbel, self).__init__()

        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, latent_dim * categorical_dim)

        self.fc4 = nn.Linear(latent_dim * categorical_dim, 256)
        self.fc5 = nn.Linear(256, 512)
        self.fc6 = nn.Linear(512, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        h2 = self.relu(self.fc2(h1))
        return self.relu(self.fc3(h2))

    def decode(self, z):
        h4 = self.relu(self.fc4(z))
        h5 = self.relu(self.fc5(h4))
        return self.sigmoid(self.fc6(h5))

    def forward(self, x, temp=0, hard=False):
        q = self.encode(x.view(-1, 784))
        q_y = q.view(q.size(0), latent_dim, categorical_dim)
        z = gumbel_softmax(q_y, temp, hard)
        return self.decode(z), F.softmax(q_y, dim=-1).reshape(*q.size())

In [21]:
latent_dim = 20
categorical_dim = 10  # one-of-K vector

temp = 1
temp_min = 0.5
ANNEAL_RATE = 0.00003

model = VAE_gumbel(temp).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, qy):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False) / x.shape[0]

    log_ratio = torch.log(qy * categorical_dim + 1e-20)
    KLD = torch.sum(qy * log_ratio, dim=-1).mean()

    return BCE + KLD

In [22]:
def train(epoch, model, train_loader, optimizer, temp, cuda=True, hard=False):
  model.train()
  train_loss = 0
  for batch_idx, (data, _) in enumerate(train_loader):
    data.to(device)
    optimizer.zero_grad()

    recon_batch, q_y = model(data, temp, hard)

    loss = loss_function(recon_batch, data, q_y)
    loss.backward()

    train_loss += loss.item() * len(data)
    optimizer.step()

    if batch_idx % 100 == 1:
        temp = np.maximum(temp * np.exp(-ANNEAL_RATE * batch_idx), temp_min)

    if batch_idx % 100 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader),
                  loss.item()))

  print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))


In [23]:
def test(epoch, model, test_loader, temp, cuda=True, hard=False):
  model.eval()
  test_loss = 0

  for i, (data, _) in enumerate(test_loader):
    data.to(device)

    recon_batch, qy = model(data, temp, hard)
    test_loss += loss_function(recon_batch, data, qy).item() * len(data)

    if i % 100 == 1:
        temp = np.maximum(temp * np.exp(-ANNEAL_RATE * i), temp_min)

    if i == 0:
        n = min(data.size(0), 8)
        comparison = torch.cat([data[:n],recon_batch.view(128, 1, 28, 28)[:n]])
        save_image(comparison.data.to(device),f"./reconstruction_{epoch:03d}.png", nrow=n)

  test_loss /= len(test_loader.dataset)
  print('====> Test set loss: {:.4f}'.format(test_loss))

In [24]:
epochs = 10
prec = math.ceil(math.log10(epochs / 100))


model = VAE_gumbel(latent_dim)
model.to(device)

VAE_gumbel(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [26]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
from torch.autograd import Variable

for epoch in range(1, epochs + 1):
    train(epoch, model, train_loader, optimizer, temp, True)
    test(epoch, model, test_loader, temp, True)

    # sample = Variable(torch.randn(64, 200)).to(device)
    # sample = model(sample).cpu()
    # save_image(sample.data.view(64, 1, 28, 28),'results/sample_' + str(epoch) + '.png')


    # M = 64 * latent_dim
    # np_y = np.zeros((M, 2), dtype=np.float32)
    # np_y[range(M), np.random.choice(2, M)] = 1
    # np_y = np.reshape(np_y, [M // latent_dim, latent_dim, 2])

    # sample = torch.from_numpy(np_y).view(M // latent_dim, latent_dim * 2)
    # sample = sample.to(device)
    # sample = model.decode(sample).to(device)

    # save_image(sample.data.view(M // latent_dim, 1, 28, 28),f"./sample_{epoch:03d}.png")

Train Epoch: 1 [0/60000 (0%)]	Loss: 186.385132
Train Epoch: 1 [12800/60000 (21%)]	Loss: 178.718109
Train Epoch: 1 [25600/60000 (43%)]	Loss: 159.979385
Train Epoch: 1 [38400/60000 (64%)]	Loss: 155.096939
Train Epoch: 1 [51200/60000 (85%)]	Loss: 148.699203
====> Epoch: 1 Average loss: 164.9116
====> Test set loss: 146.7252
Train Epoch: 2 [0/60000 (0%)]	Loss: 141.607956
Train Epoch: 2 [12800/60000 (21%)]	Loss: 136.666580
Train Epoch: 2 [25600/60000 (43%)]	Loss: 141.950455
Train Epoch: 2 [38400/60000 (64%)]	Loss: 133.401276
Train Epoch: 2 [51200/60000 (85%)]	Loss: 128.395447
====> Epoch: 2 Average loss: 139.8862
====> Test set loss: 134.7699
Train Epoch: 3 [0/60000 (0%)]	Loss: 137.288391
Train Epoch: 3 [12800/60000 (21%)]	Loss: 131.569946
Train Epoch: 3 [25600/60000 (43%)]	Loss: 138.998703
Train Epoch: 3 [38400/60000 (64%)]	Loss: 135.929230
Train Epoch: 3 [51200/60000 (85%)]	Loss: 132.329407
====> Epoch: 3 Average loss: 131.8911
====> Test set loss: 128.9055
Train Epoch: 4 [0/60000 (0%)]	L